In [2]:
!pip install openai requests tiktoken numpy


[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: C:\Users\dhmore\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('sample.env', override=True)


True

In [7]:
# Run preliminary checks

import json

# Load the training set
with open('train-dataset.jsonl', 'r', encoding='utf-8') as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Number of examples in training set:", len(training_dataset))
print("First example in training set:")
for message in training_dataset[0]["messages"]:
    print(message)

# Load the validation set
with open('validation-dataset.jsonl', 'r', encoding='utf-8') as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNumber of examples in validation set:", len(validation_dataset))
print("First example in validation set:")
for message in validation_dataset[0]["messages"]:
    print(message)

Number of examples in training set: 1500
First example in training set:
{'role': 'system', 'content': '\nYou are an assistant that is an expert in generating sqlite SQL queries.\nHaving the access to database content, generate a correct sqlite SQL query for the given question.\n### Database content ###\n        \nCREATE TABLE trip (\n\tid INTEGER, \n\tduration INTEGER, \n\tstart_date TEXT, \n\tstart_station_name TEXT, \n\tstart_station_id INTEGER, \n\tend_date TEXT, \n\tend_station_name TEXT, \n\tend_station_id INTEGER, \n\tbike_id INTEGER, \n\tsubscription_type TEXT, \n\tzip_code INTEGER, \n\tPRIMARY KEY (id)\n)\n/*\nColumns in trip and 3 examples in each column for high cardinality columns :\nid : 900645, 900752, 900524\nduration : 1131, 2146, 1155\nstart_date : 8/21/2015 17:39, 8/21/2015 17:03, 8/21/2015 17:16\nstart_station_name : Howard at 2nd, 2nd at Folsom, Market at 10th\nstart_station_id : 56, 65, 49\nend_date : 8/21/2015 17:19, 8/21/2015 18:08, 8/21/2015 17:32\nend_station_na

In [8]:
# Validate token counts
#!pip install tiktoken -U
import json
import tiktoken
import numpy as np
from collections import defaultdict

#encoding = tiktoken.get_encoding("o200k_base") 
encoding = tiktoken.get_encoding("cl100k_base") 
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

files = ['train-dataset.jsonl', 'validation-dataset.jsonl']

for file in files:
    print(f"Processing file: {file}")
    with open(file, 'r', encoding='utf-8') as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

    print_distribution(total_tokens, "total tokens")
    print_distribution(assistant_tokens, "assistant tokens")
    print('*' * 50)

Processing file: train-dataset.jsonl

#### Distribution of total tokens:
min / max: 3331, 4018
mean / median: 3891.806666666667, 3913.0
p5 / p95: 3761.9, 3996.0

#### Distribution of assistant tokens:
min / max: 44, 206
mean / median: 77.59133333333334, 65.0
p5 / p95: 50.0, 119.0
**************************************************
Processing file: validation-dataset.jsonl

#### Distribution of total tokens:
min / max: 3219, 4017
mean / median: 3888.842, 3902.0
p5 / p95: 3771.9, 3996.0

#### Distribution of assistant tokens:
min / max: 44, 203
mean / median: 80.744, 66.0
p5 / p95: 47.900000000000006, 126.10000000000002
**************************************************


In [9]:
# Upload fine-tuning files

import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version = "2024-08-01-preview"  # This API version or later is required to access seed/events/checkpoint features
)

training_file_name = 'train-dataset.jsonl'
validation_file_name = 'validation-dataset.jsonl'

# Upload the training and validation dataset files to Azure OpenAI with the SDK.

training_response = client.files.create(
    file = open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file = open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)


Training file ID: file-18e6317e597e43969229581e0348990b
Validation file ID: file-628fff7b9d2c46a193a011d7f49e094e


In [10]:
# Submit fine-tuning training job

response = client.fine_tuning.jobs.create(
    training_file = training_file_id,
    validation_file = validation_file_id,
    model = "gpt-4o-mini-2024-07-18", # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters.
    seed = 100 # seed parameter controls reproducibility of the fine-tuning job. If no seed is specified one will be generated automatically.
)

job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.

print("Job ID:", response.id)
print("Status:", response.status)
print(response.model_dump_json(indent=2))

Job ID: ftjob-a58f137fe81f4c09ba7a65b769ff2350
Status: pending
{
  "id": "ftjob-a58f137fe81f4c09ba7a65b769ff2350",
  "created_at": 1727329216,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": -1,
    "batch_size": -1,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-4o-mini-2024-07-18",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "seed": 100,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-18e6317e597e43969229581e0348990b",
  "validation_file": "file-628fff7b9d2c46a193a011d7f49e094e",
  "estimated_finish": 1730094916,
  "integrations": null
}


In [12]:
# Track training status

from IPython.display import clear_output
import time

start_time = time.time()

# Get the status of our fine-tuning job.
response = client.fine_tuning.jobs.retrieve(job_id)

status = response.status

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)

    response = client.fine_tuning.jobs.retrieve(job_id)
    print(response.model_dump_json(indent=2))
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = response.status
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Fine-tuning job {job_id} finished with status: {status}')

# List all fine-tuning jobs for this resource.
print('Checking other fine-tune jobs for this resource.')
response = client.fine_tuning.jobs.list()
print(f'Found {len(response.data)} fine-tune jobs.')

Fine-tuning job ftjob-a58f137fe81f4c09ba7a65b769ff2350 finished with status: succeeded
Checking other fine-tune jobs for this resource.
Found 3 fine-tune jobs.


In [13]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10)
print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "ftevent-34da367f1ef04d6da93cdf1c417e8d35",
      "created_at": 1727335406,
      "level": "info",
      "message": "Training tokens billed: 52261000",
      "object": "fine_tuning.job.event",
      "type": "message"
    },
    {
      "id": "ftevent-f4f9ea07f811468b8e9aa3939b4a6c3f",
      "created_at": 1727335406,
      "level": "info",
      "message": "Completed results file: file-69da9b16353649bfa7c0e9c87cff60ca",
      "object": "fine_tuning.job.event",
      "type": "message"
    },
    {
      "id": "ftevent-42aeb4994b484b62991feb6bbdff4566",
      "created_at": 1727335405,
      "level": "info",
      "message": "Model Evaluation Passed.",
      "object": "fine_tuning.job.event",
      "type": "message"
    },
    {
      "id": "ftevent-c8f048c120d94b1e91b45e936e055063",
      "created_at": 1727335397,
      "level": "info",
      "message": "Postprocessing started (retry 2).",
      "object": "fine_tuning.job.event",
      "type": "message"
   

In [14]:
response = client.fine_tuning.jobs.checkpoints.list(job_id)
print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "ftchkpt-7290aa69b7c74dd699a1f6adabac88cd",
      "created_at": 1727334843,
      "fine_tuned_model_checkpoint": "gpt-4o-mini-2024-07-18.ft-a58f137fe81f4c09ba7a65b769ff2350",
      "fine_tuning_job_id": "ftjob-a58f137fe81f4c09ba7a65b769ff2350",
      "metrics": {
        "full_valid_loss": 0.07230913800506274,
        "full_valid_mean_token_accuracy": 0.6882239139720108,
        "step": 1500.0,
        "train_loss": 0.00575849087908864,
        "train_mean_token_accuracy": 1.0,
        "valid_loss": 0.09541347414948219,
        "valid_mean_token_accuracy": 0.9709302325581395
      },
      "object": "fine_tuning.job.checkpoint",
      "step_number": 1500
    },
    {
      "id": "ftchkpt-9d26f3db1c034e4fb352c0f349345da9",
      "created_at": 1727333709,
      "fine_tuned_model_checkpoint": "gpt-4o-mini-2024-07-18.ft-a58f137fe81f4c09ba7a65b769ff2350:ckpt-step-1000",
      "fine_tuning_job_id": "ftjob-a58f137fe81f4c09ba7a65b769ff2350",
      "metrics": {
 

In [ ]:
# Deploy fine-tuned model

import json
import requests

token = os.getenv("TEMP_AUTH_TOKEN")
subscription = "<YOUR_SUBSCRIPTION_ID>"
resource_group = "<YOUR_RESOURCE_GROUP_NAME>"
resource_name = "<YOUR_AZURE_OPENAI_RESOURCE_NAME>"
model_deployment_name = "gpt-4o-mini-2024-07-18-ft" # Custom deployment name you chose for your fine-tuning model

deploy_params = {'api-version': "2023-05-01"}
deploy_headers = {'Authorization': 'Bearer {}'.format(token), 'Content-Type': 'application/json'}

deploy_data = {
    "sku": {"name": "standard", "capacity": 1},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": "<YOUR_FINE_TUNED_MODEL>", #retrieve this value from the previous call, it will look like gpt-4o-mini-2024-07-18.ft-0e208cf33a6a466994aff31a08aba678
            "version": "1"
        }
    }
}
deploy_data = json.dumps(deploy_data)

request_url = f'https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}'

print('Creating a new deployment...')

r = requests.put(request_url, params=deploy_params, headers=deploy_headers, data=deploy_data)

print(r)
print(r.reason)
print(r.json())

In [3]:
# Use the deployed customized model

import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version = "2024-06-01"
)

response = client.chat.completions.create(
    model = "nl2sql-ft-gpt-4o-mini-2024-07-18-", # model = "Custom deployment name you chose for your fine-tuning model"
    messages = [
        {"role": "system", "content": "You are an assistant that is an expert in generating sqlite SQL queries.\nHaving the access to database content, generate a correct sqlite SQL query for the given question."},
        {"role": "user", "content": "How many products are never booked with amount higher than 200?"}
    ]
)

print(response.choices[0].message.content)

To generate the query, I'll be working with the following tables: PRODUCT, Orders.
From these tables, I'll be using the following columns: PRODUCT.id, Orders.product_id, Orders.amount.
The SQL query I'll be generating is:
```sql
SELECT count(*) FROM PRODUCT WHERE id NOT IN (SELECT product_id FROM Orders WHERE amount > 200)
```
